In [2]:
import os
import sys
import requests
import pandas
import paramiko
import json
from IPython import display
from pathlib import Path
import configparser

In [3]:
from curation_common import *
from htsworkflow.submission.encoded import DCCValidator

In [4]:
from htsworkflow.submission.encoded import Document
from htsworkflow.submission.aws_submission import run_aws_cp
from htsworkflow.util.api import (
    add_auth_options,
    make_auth_from_opts,
    HtswApi,
)

In [5]:
config = configparser.ConfigParser()
config.read([os.path.expanduser('~/.htsworkflow.ini'),
             '/etc/htsworkflow.ini'
             ])

SECTION = 'sequence_archive'
if config.has_section(SECTION):
    apiid = config.get(SECTION, 'apiid')
    apikey = config.get(SECTION, 'apikey')
    apihost = config.get(SECTION, 'host')

apihost='http://jumpgate.caltech.edu'    
auth = {'apiid': apiid, 'apikey': apikey }
htsw = HtswApi(apihost, auth)

In [6]:
# live server & control file
server = ENCODED('www.encodeproject.org')
server.load_netrc()

'/home/diane/proj/encode4-curation'

In [7]:
graph = server.get_json('https://www.encodeproject.org/search/?type=Experiment&@id=%2Fexperiments%2FENCSR479MNN%2F&@id=%2Fexperiments%2FENCSR288RRZ%2F&@id=%2Fexperiments%2FENCSR899OKE%2F&@id=%2Fexperiments%2FENCSR464VSR%2F&@id=%2Fexperiments%2FENCSR774MGO%2F&@id=%2Fexperiments%2FENCSR129VBC%2F&@id=%2Fexperiments%2FENCSR420YFF%2F&@id=%2Fexperiments%2FENCSR942YMN%2F&@id=%2Fexperiments%2FENCSR648YUM%2F&@id=%2Fexperiments%2FENCSR903XMI%2F&@id=%2Fexperiments%2FENCSR244HHV%2F&@id=%2Fexperiments%2FENCSR168PXI%2F&@id=%2Fexperiments%2FENCSR308XAR%2F&@id=%2Fexperiments%2FENCSR484WZL%2F&@id=%2Fexperiments%2FENCSR450EXF%2F')

Error http status: 400 for https://www.encodeproject.org/search/?type=Experiments&lab.title=Barbara+Wold%2C+Caltech&award.rfa=ENCODE4


HTTPError: 400 Client Error: Bad Request for url: https://www.encodeproject.org/search/?type=Experiments&lab.title=Barbara+Wold%2C+Caltech&award.rfa=ENCODE4&limit=all

In [19]:
for row in graph['@graph']:
    for replicate_sparse in row['replicates']:
        replicate = server.get_json(replicate_sparse['@id'])
        library_id = replicate['library']['@id']
        fragment_size = replicate['library'].get('average_fragment_size')
        jumpgate_id = replicate['library']['aliases'][0].split(':')[1]
        if not jumpgate_id.startswith('SL'):
            jumpgate_info = htsw.get_library(jumpgate_id)
        else:
            jumpgate_info = {'insert_size': 'unknown'}
        print(library_id, jumpgate_id, fragment_size, jumpgate_info['insert_size'])

/libraries/ENCLB492DNB/ 22576 265 265
/libraries/ENCLB641ESO/ 22578 259 259
/libraries/ENCLB706JRN/ 22577 260 260
/libraries/ENCLB952BRY/ 22569 250 250
/libraries/ENCLB404DIP/ 22574 256 256
/libraries/ENCLB693TEO/ 22575 267 267
/libraries/ENCLB445WKP/ 22570 246 246
/libraries/ENCLB081RMC/ 22568 255 255
/libraries/ENCLB603CZI/ 22565 250 250
/libraries/ENCLB979UQX/ 22567 254 254
/libraries/ENCLB462YMC/ 22573 269 269
/libraries/ENCLB542TEM/ 22563 251 251
/libraries/ENCLB784DQM/ 22566 255 255
/libraries/ENCLB758ADE/ 22572 255 255
/libraries/ENCLB480EGW/ SL428074_E3 244 unknown
/libraries/ENCLB045XLK/ SL428072_E1 242 unknown
/libraries/ENCLB872CWU/ SL428073_E2 242 unknown
/libraries/ENCLB396RKH/ 22564 246 246
/libraries/ENCLB076XKH/ 22581 243 243
/libraries/ENCLB657TWS/ 22580 246 246
/libraries/ENCLB556TQU/ 22579 247 247
/libraries/ENCLB136WOV/ 22571 264 264


In [18]:
jumpgate_id

'SL428074_E3'